# Problem Set 7 (Total points: 100)


### Q1. Markov Decision Process [20 points]
Imagine an MDP corresponding to playing slot machines in a casino. Suppose you start with \\$20 cash to spend in the casino,  and you decide to play until you lose all your money or until you double your money (i.e., increase your cash to at least \\$40). There are two slot machines you can choose to play: 1) slot machine X costs \\$10 to play and will pay out \\$20 with probability 0.05 and will pay \\$0 otherwise;
and 2) slot machine Y costs \\$20 to play and will pay out \\$30 with probability 0.01 and \\$0 otherwise. As you are playing, you keep choosing machine X or Y at each turn.

Write down the MDP that corresponds to the above problem. Clearly specify the state space, action space, rewards and transition probabilities. Indicate which state(s) are terminal. Assume that the discount factor γ = 1.

**Notes:** There are several valid ways to specify the MDP, so you have some flexibility in your solution. For example, rewards can take many different forms, but overall you should get a higher reward for stopping when you double your money than when you lose all your money!

***Solution***:

State Space S:

{no cash left, $\$$10 cash left, $\$$20 cash left, $\$$30 cash left, $\$$40 cash left}.


The MDP will be over when the state is "no cash left" or "$\$$40 cash left".


Action Space A: 

{Choose to play slot machine X, Choose to play slot machine Y}.

Reward R:

R($\$$10 cash left, choose to play X, no cash left) = -50

R($\$$10 cash left, choose to play X, $\$$20 cash left) = 10  

R($\$$20 cash left, choose to play X, $\$$10 cash left) = -10 

R($\$$20 cash left, choose to play X, $\$$30 cash left) = 10

R($\$$20 cash left, choose to play Y, no cash left) = -50

R($\$$20 cash left, choose to play Y, $\$$30 cash left) = 10

R($\$$30 cash left, choose to play X, $\$$20 cash left) = -10

R($\$$30 cash left, choose to play X, $\$$40 cash left) = 100

R($\$$30 cash left, choose to play Y, $\$$10 cash left) = -10

R($\$$30 cash left, choose to play Y, $\$$40 cash left) = 100

Transition Probability $P_{sa}$:

P($\$$10 cash left, choose to play X, no cash left) = 0.95 

P($\$$10 cash left, choose to play X, $\$$20 cash left) = 0.05 

P($\$$20 cash left, choose to play X, $\$$10 cash left) = 0.95 

P($\$$20 cash left, choose to play X, $\$$30 cash left) = 0.05

P($\$$20 cash left, choose to play Y, no cash left) = 0.99

P($\$$20 cash left, choose to play Y, $\$$30 cash left) = 0.01

P($\$$30 cash left, choose to play X, $\$$20 cash left) = 0.95 

P($\$$30 cash left, choose to play X, $\$$40 cash left) = 0.05

P($\$$30 cash left, choose to play Y, $\$$10 cash left) = 0.99

P($\$$30 cash left, choose to play Y, $\$$40 cash left) = 0.01


## Reinforcement Learning
In the remainder of the problem set you will implement the Q-Learning Algorithm to solve the "Frozen Lake" problem.
We​ ​will​ ​use​ ​OpenAI’s​ ​gym​ ​package​ ​to​ ​develop​ ​our​ ​solution​ ​in​ ​Python.​ 

### OpenAI Gym Setup 
​Read the​ ​set-up​ ​instructions for​ ​Gym​  [here](https://gym.openai.com/docs/).​ ​The​ ​instructions​ ​also​ ​give​ ​a​ ​good​ ​overview​ ​of​ ​the​ ​API​ ​for​ ​this​ ​package,​ ​so​ ​please​ ​read through​ ​it​ ​before​ ​proceeding.​



### Frozen Lake
Instead​ ​of​ ​using​ ​CartPole,​ ​we’re​ ​going​ ​to​ ​be​ ​using​ ​the​ [​​FrozenLake](https://gym.openai.com/envs/FrozenLake-v0/) environment. Read through the code for this environment by following the Github link.​ <br>

Winter​ ​is​ ​quickly​ ​approaching,​ ​and​ ​we​ ​have​ ​to​ ​worry​ ​about​ ​navigating​ ​frozen​ ​lakes.​ ​It’s​ ​only early November,​ ​
so​ ​the​ ​lakes​ ​haven’t​ ​completely​ ​frozen​ ​and​ ​if​ ​you​ ​make​ ​the​ ​wrong​ ​step​ ​you​ ​may​ ​fall​ ​through. 
We’ll​ ​need​ ​to​ ​learn​ ​how​ ​to​ ​get​ ​to​ ​our​ ​destination​ ​when​ ​stuck​ ​on​ ​the​ ​ice,​ ​without​ ​falling​ ​in.
The​ ​lake​ ​we’re​ ​going​ ​to​ ​consider​ ​is a​ ​square​ ​lake​ ​with​ ​spots​ ​to​ ​step​ ​on​ ​in​ ​the​ ​shape​ ​of​ ​a​ ​grid.​ ​​<br>

The surface is described using a 4x4 grid like the following

        S F F F 
        F H F H
        F F F H
        H F F G

​Each​ ​spot​ ​can have​ ​one​ ​of​ ​four​ ​states:
- S:​ ​starting​ ​point.
- G:​ ​goal​ ​point.
- F:​ ​frozen​ ​spot,​ ​where​ ​it’s​ ​safe​ ​to​ ​walk.
- H:​ ​hole​ ​in​ ​the​ ​ice,​ ​where​ ​it’s​ ​not​ ​safe​ ​to​ ​walk.<br>


For example, consider the lake, <br>
![alt text](https://cs-people.bu.edu/sunxm/frozen_lake_example.png)

There are four possible actions: UP, DOWN, LEFT, RIGHT. Although we​ ​can​ ​see​ ​the​ ​path​ ​we​ ​need​ ​to​ ​walk,​ the agent does not. ​We’re​ ​going​ ​to​ ​train​ ​an​ ​agent​ ​to​ ​discover​ ​this​ ​via​ ​problem solving.​ ​However,​ ​walking​ ​on​ ​ice​ ​isn’t​ ​so​ ​easy!​ ​Sometimes​ ​you​ ​slip​ ​and​ ​aren’t​ ​able​ ​to​ ​take​ ​the​ ​step​ ​you intended.

The episode ends when you reach the goal or fall in a hole.

You receive a reward of 1 if you reach the goal, and zero otherwise.

### Q2. Walking on the Frozen Lake [10 points]

Write a script that sets up the Frozen Lake environment and takes 10 walks through it, consisting of a maximum 10 randomly sampled actions during each walk. After each step, render the current state, and print out the reward and whether the walk is "done", i.e. in the terminal state, because the agent fell into a hole (stop if it is). In your own words, explain how this environment behaves. 

In [1]:
pip install gym

Note: you may need to restart the kernel to use updated packages.


In [2]:
import gym
import numpy as np
env = gym.make('FrozenLake-v1').unwrapped
env.reset()
for i_episode in range(10):
    observation = env.reset()
    for j in range(10):
        env.render()
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action) 
        print("Reward:", reward)
        print("Done:", done)
        if done:
            break;
env.close()


SFFF
FHFH
FFFH
HFFG
Reward: 0.0
Done: False
  (Down)
SFFF
FHFH
FFFH
HFFG
Reward: 0.0
Done: True

SFFF
FHFH
FFFH
HFFG
Reward: 0.0
Done: False
  (Right)
SFFF
FHFH
FFFH
HFFG
Reward: 0.0
Done: False
  (Up)
SFFF
FHFH
FFFH
HFFG
Reward: 0.0
Done: False
  (Left)
SFFF
FHFH
FFFH
HFFG
Reward: 0.0
Done: False
  (Left)
SFFF
FHFH
FFFH
HFFG
Reward: 0.0
Done: False
  (Up)
SFFF
FHFH
FFFH
HFFG
Reward: 0.0
Done: True

SFFF
FHFH
FFFH
HFFG
Reward: 0.0
Done: False
  (Up)
SFFF
FHFH
FFFH
HFFG
Reward: 0.0
Done: False
  (Down)
SFFF
FHFH
FFFH
HFFG
Reward: 0.0
Done: False
  (Right)
SFFF
FHFH
FFFH
HFFG
Reward: 0.0
Done: True

SFFF
FHFH
FFFH
HFFG
Reward: 0.0
Done: False
  (Down)
SFFF
FHFH
FFFH
HFFG
Reward: 0.0
Done: False
  (Right)
SFFF
FHFH
FFFH
HFFG
Reward: 0.0
Done: True

SFFF
FHFH
FFFH
HFFG
Reward: 0.0
Done: False
  (Down)
SFFF
FHFH
FFFH
HFFG
Reward: 0.0
Done: False
  (Left)
SFFF
FHFH
FFFH
HFFG
Reward: 0.0
Done: False
  (Down)
SFFF
FHFH
FFFH
HFFG
Reward: 0.0
Done: False
  (Left)
SFFF
FHFH
FFFH
HFFG
Reward: 0.0

### Q3. Q-Learning [50 points]

You will ​implement​ ​Q-learning to solve the problem.​ Assume that the environment has states S and actions A. Use the ​function​ ​signature​ ​provided​ below:

``` python
def​ q_learning(env,​ ​alpha=0.5,​ gamma=0.95,​ ​epsilon=0.1, num_episodes=500):
""" ​Performs​ ​Q-learning​ ​for​ ​the​ ​given​ ​environment.
Initialize​ ​Q​ ​to​ ​all​ ​zeros.
:param​ ​env:​ ​Unwrapped​ ​OpenAI​ ​gym​ ​environment.
:param​ ​alpha:​ ​Learning​ ​rate​ ​parameter.
:param​ ​gamma:​ ​Decay​ ​rate​ (future reward discount) ​parameter.
:param​ ​num_episodes:​ ​Number​ ​of​ ​episodes​ ​to​ ​use​ ​for​ ​learning. 
:return:​ ​Q​ ​table, i.e. a table with the Q value for every <S, A> pair.
"""
```


The pseudocode for Q-Learning was described in lecture, but for your reference, we provide it here:
![alt text](https://cs-people.bu.edu/sunxm/q-learning.png)


In [3]:
def q_learning(env, alpha=0.5, gamma=0.95, epsilon=0.1, num_episodes=500, random_init = False):
    np.random.seed(42)

    if random_init == False:
        q_table = np.zeros((env.observation_space.n, env.action_space.n))
    else:    
        q_table = np.random.rand(env.observation_space.n, env.action_space.n)

    for i in range(num_episodes):
        done = False
        state = env.reset()
        
        while not done:
            prob = np.random.uniform(0,1)
            if prob > epsilon:
                if q_table[state,0] == q_table[state,1] == q_table[state,2] == q_table[state,3]:
                    action = int(np.random.randint(4,size = 1))
                else:
                    action = np.argmax(q_table[state,:])
            else:
                action = env.action_space.sample()
                
            new_state, reward, done, info = env.step(action) 
            max_Q = np.max(q_table[new_state,:])
            q_table[state][action] = q_table[state][action] + alpha * ((reward + (gamma * max_Q) - q_table[state][action]))
            state = new_state
            
    return q_table

### Q4. Main Function [20 points]
You also need to implement the main function to solve the entire FrozenLake-v0 problem, including setting up the Gym environment, 
calling the q-learning function as defined above, printing out the returned Q-table, etc. <br>

You should use the $\epsilon$-greedy algorithm (solving Exploration-Exploitation Dilemma) to generate actions for each state. Try `num_episodes` with different values, e.g. `num_episodes=500, 1000, 5000, 10000, 50000`. You should also try different initializations for the Q-table, i.e. Random Initialization and Zero Initialization. 

Please do not change any default value of environment setting, such as `is_slippery`, if not mentioned above.

Provide​ ​the​ ​final​ ​Q-table​ ​of​ ​​FrozenLake-v0​ for **each <num_episode, init_method>** you have tried [10 points],​ ​and analyze ​what​ you observe [10 points]. 

If you are interested in testing your learned Q-learned, you can compute the win rate (the probability of agents to goal following the Q-table). Note that you should use `argmax` instead of epsilon greedy to choose the action at the current state. 
Win Rate is a good way to test whether you write your algorithm correctly.

Just for your reference, we test our implementation for 10000 trials after 50000-episode training using zero initialization and the win rate is 0.4985. You should achieve similar value under the same test.



In [4]:
num_episode = [500, 1000, 5000, 10000, 50000]

env = gym.make('FrozenLake-v1').unwrapped

print("Initialize the Q-table with zeros")
for i in num_episode:  
    print("The number of episodes:", i)
    print(q_learning(env, 0.5, 0.95, 0.1, i, False))
  
print("Initialize the Q-table with random number between 0 and 1")
for j in num_episode: 
    print("The number of episodes:", j)
    print(q_learning(env, 0.5, 0.95, 0.1, j, True))

Initialize the Q-table with zeros
The number of episodes: 500
[[0.15824509 0.10136999 0.12086642 0.11701772]
 [0.06022394 0.03911476 0.01309797 0.09769829]
 [0.04535202 0.0520334  0.04628593 0.05773853]
 [0.0344887  0.04124574 0.03526499 0.05063952]
 [0.20384944 0.07070142 0.035757   0.03660859]
 [0.         0.         0.         0.        ]
 [0.0712186  0.01421652 0.01776473 0.00690219]
 [0.         0.         0.         0.        ]
 [0.19228369 0.10167316 0.16181201 0.29650994]
 [0.12668874 0.39300645 0.06140277 0.17213502]
 [0.22334252 0.0780073  0.         0.00981819]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.19242893 0.16306879 0.68723293 0.50597705]
 [0.62470577 0.61907223 0.52201677 0.8495072 ]
 [0.         0.         0.         0.        ]]
The number of episodes: 1000
[[0.15257714 0.11906468 0.14514301 0.13035266]
 [0.0573819  0.05553607 0.02828044 0.08818235]
 [0.06581191 0.06055998 0.06331068 0.07797309]
 [0.04856126 0.

#### Additional Instructions
If​ ​you’re​ ​new​ ​to​ OpenAI’s​ ​Gym,​ first ​go​ ​through​ ​OpenAI’s​ ​full​ ​tutorial listed​ ​earlier​ ​and​ ​visit​ ​the​ ​Appendix​ ​to​ ​this​ ​homework​ ​before​ ​proceeding.
Some additional rules:
- Only submit **original code** written entirely by you.
- **Permitted​​ non-standard ​​libraries**:​​ ​gym​,​​ ​numpy.
- **Only​ ​use​ ​numpy​ ​for​ ​random​ ​sampling,​ ​and​ ​seed​ ​at​ ​the​ ​beginning​ ​of​ ​each​ ​function​ ​with**:
np.random.seed(42)
- **Unwrap​ ​the​ ​OpenAI​ ​gym​ ​before​ ​providing​ ​them​ ​to​ ​these​ ​functions.** <br>
env​ ​=​ ​gym.make(“FrozenLake-v0”).unwrapped


## Appendix

This​ ​appendix​ ​includes​ ​references​ ​to​ ​APIs​ ​you​ ​may​ ​find​ ​useful.​ ​If​ ​the​ ​description​ ​sounds​ ​useful,​ ​check out
the​ ​respective​ ​package’s​ ​documentation​ ​for​ ​a​ ​much​ ​better​ ​description​ ​than​ ​we​ ​could​ ​provide.

#### Numpy
- np.zeros:​ ​N-dimensional​ ​tensor​ ​initialized​ ​to​ ​all​ ​0s.
- np.ones:​ ​N-dimensional​ ​tensor​ ​initialized​ ​to​ ​all​ ​1s.
- np.eye:​ ​N-dimensional​ ​tensor​ ​initialized​ ​to​ ​a​ ​diagonal​ ​matrix​ ​of​ ​1s.
- np.random.choice:​ ​Randomly​ ​sample​ ​from​ ​a​ ​list,​ ​allowing​ ​you​ ​to​ ​specify​ ​weights.
- np.argmax:​ ​Index​ ​of​ ​the​ ​maximum​ ​element.
- np.abs:​ ​Absolute​ ​value.
- np.mean:​ ​Average​ ​across​ ​dimensions.
- np.sum:​ ​Sum​ ​across​ ​dimensions.

### OpenAI Gym
- Environment​ ​(unwrapped):<br>
env.nS #​ ​Number​ ​of​ ​spaces. <br>
env.nA #​ ​Number​ ​of​ ​actions. <br>
env.P #​ ​Dynamics​ ​model.
